# BLU10 - Learning Notebook - Appendix

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy.sparse import random, coo_matrix, lil_matrix, dok_matrix, csr_matrix, csc_matrix
from statistics import mean
from math import nan, isnan

# A More on Sparse Matrices


## A.1 Implementation of sparse matrices in SciPy

Let's explore how sparse matrices work and exemplify some implementations

### A.1.1 Dictionary of Keys (DOK)

The most straightforward implementation of a sparse matrix is as a dictionary of keys, in which the keys are tuples that represent indices.

```
┌───┬───┬───┐          
│ 1 │ 0 │ 0 │          {  
├───┼───┼───┤            (0, 0): 1,
│ 0 │ 1 │ 0 │ → DoK →    (1, 1): 1,
├───┼───┼───┤            (2, 2): 1,
│ 0 │ 0 │ 1 │          }
└───┴───┴───┘ 
```

In [2]:
B = random(5, 5, density=.04, format='dok', random_state=42)

B.toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.30424224, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

In [3]:
dict(B)

{(3, 1): 0.3042422429595377}

### A.1.2 Linked List (LIL)

The row-based linked list format uses two NumPy arrays with regular Python lists inside them, one list per row.

The first array stores the values in the cells ordered from the left-most column to the right-most one. (If there's no data, the row is an empty list.)

The second stores which cells are occupied, i.e., the column index of the values.

```
┌───┬───┬───┐         Matrix data:    Matrix rows:          
│ 1 │ 0 │ 1 │          [               [  
├───┼───┼───┤            [1, 1],         [0, 2],
│ 0 │ 0 │ 0 │ → LIL →    [],             [],
├───┼───┼───┤            [1],            [2]
│ 0 │ 0 │ 1 │          ]               ]
└───┴───┴───┘ 
```

In [4]:
C = random(5, 5, density=.04, format='lil', random_state=45)

C.toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.97600332, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

In [5]:
C.data

array([list([]), list([0.9760033158175416]), list([]), list([]), list([])],
      dtype=object)

In [6]:
C.rows

array([list([]), list([3]), list([]), list([]), list([])], dtype=object)

### A.1.3 Coordinate List (COO)

A `COO` matrix uses three NumPy arrays, and for each occupied cell, there is a value in the three of them.

Perhaps without surprise, the first stores the values, the second the row indexes and the third the column indexes.

```
┌───┬───┬───┐                   
│ 1 │ 0 │ 1 │          Matrix data: [1, 1, 1] 
├───┼───┼───┤          
│ 0 │ 0 │ 0 │ → COO →  Matrix rows: [0, 0, 2]
├───┼───┼───┤          
│ 0 │ 0 │ 1 │          Matrix cols: [0, 2, 2]
└───┴───┴───┘ 
```

In [7]:
D = random(5, 5, density=.04, format='coo', random_state=54)

D.toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.92627396],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

In [8]:
D.data

array([0.92627396])

In [9]:
D.row

array([3])

In [10]:
D.col

array([4], dtype=int32)

### A.1.4 Compressed Sparse (CS)

The **Compressed Sparse Row (CSR)**, uses three arrays:
* `data`, the value vector containing all non-zero values in [row-major order](https://en.wikipedia.org/wiki/Row-_and_column-major_order)
* `indptr`, the index pointer indicates at which element of the value vector the row starts
* `indices`, contains the column indices (which column each of the values come from).

```
┌───┬───┬───┐                   
│ 1 │ 0 │ 1 │          Matrix data:    [1, 1, 1] 
├───┼───┼───┤          
│ 0 │ 0 │ 0 │ → CSR →  Matrix indptr:  [0, 2, 2, 4]
├───┼───┼───┤          
│ 0 │ 0 │ 1 │          Matrix indices: [0, 2, 2]
└───┴───┴───┘ 
```

In fact, the index pointers tell us the starting and stopping indices `data[i, j]` for each row, above:
* The first row is given by `data[0:2]`
* The second row is given by `data[2:2]`
* The third row is given by `data[2:4]`.

For a better visualization, check the following CSR representation, which can be found [here](https://matteding.github.io/2019/04/25/sparse-matrices/):

<img src="https://matteding.github.io/images/csr.gif" width="500"></img>

The **Compressed Sparse Column (CSC)** format is similar, but the pointers refer to columns and the indices to the rows.

When comparing the two types of Compressed Sparse matrices:
* `CSR` provides efficient row slicing but slow column slicing, i.e., accessing and operating on row vectors
* `CSC` provides efficient column slicing but slow row slicing, i.e., accessing and operating on column vectors.


In [11]:
E = random(5, 5, density=.2, format='csr', random_state=65)

E.toarray()

array([[0.22027153, 0.        , 0.        , 0.16514066, 0.        ],
       [0.        , 0.        , 0.73870729, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.92758757, 0.        , 0.        , 0.        , 0.        ],
       [0.10069371, 0.        , 0.        , 0.        , 0.        ]])

In [12]:
E.data

array([0.22027153, 0.16514066, 0.73870729, 0.92758757, 0.10069371])

In [13]:
E.indptr

array([0, 2, 3, 3, 4, 5], dtype=int32)

In [14]:
E.indices

array([0, 3, 2, 0, 0], dtype=int32)

### A.1.5 Summary

The main takeaways (aka the sparse matrices guide to happiness):

* `DOK`, `LIL` and `COO` are all convenient formats to construct sparse matrices
* `COO` is a fast format for constructing sparse matrices and is preferred to `DOK` and `LIL` for large matrices
* On the other hand, `DOK` and `LIL` deal efficiently with changes to the sparsity structure, suited for incremental construction
* Once we construct the matrix, we convert it to `CSR` or `CSC` fast operations, the most used for write-once-read-many tasks.

| Sparse Matrix Type       | Street Name | Good                                                                                       | Bad                                                                   |
|--------------------------|-------------|--------------------------------------------------------------------------------------------|-----------------------------------------------------------------------|
| Dictionary of Keys (DOK) | DOK         | Fast incremental construction; Flexible structure; Efficient access to individual elements | Slow iteration; Slow arithmetics; Slow slicing                        |
| List of Lists (LIL)      | LIL         | Fast incremental construction; Flexible structure                                          | Slow arithmetics; Slow column slicing; Slow matrix-vector products    |
| Coordinate List (COO)    | COO         | Fast conversion to other sparse formats; Permits duplicate entries                         | Doesn't support arithmetic operations; Doesn't support slicing        |
| Compressed Sparse (CS)   | CSR/CSC     | Changes to sparsity structure are expensive                                                | Efficient arithmetics; Efficient slicing; Fast matrix-vector products |

*Table 1: Comparison of the main sparse matrix types in SciPy.*


## A.2 Creating Sparse Matrices

Back to our rating matrix $R$ from the previous unit, as:

```
    ┌───┬───┬───┐                   
    │ 1 │   │ 2 │
    ├───┼───┼───┤          
R = │ 1 │ 5 │   │
    ├───┼───┼───┤          
    │   │ 2 │ 1 │
    └───┴───┴───┘ 
```

In this section, we build sparse representations of $R$.

We start from our standard array.

In [15]:
data = np.array([1, 0, 2, 1, 5, 0, 0, 2, 1]).reshape(3, 3)
data

array([[1, 0, 2],
       [1, 5, 0],
       [0, 2, 1]])

### A.2.1 DOK and LIL

The use-case for both `DOK` and `LIL` is incremental construction.

In [16]:
F = dok_matrix((3, 3))

for i in range(3):
    for j in range(3):
        F[i, j] = data[i, j]

F.toarray()

array([[1., 0., 2.],
       [1., 5., 0.],
       [0., 2., 1.]])

In [17]:
G = lil_matrix((3, 3))

G[data.nonzero()] = data[data.nonzero()]

G.toarray()

array([[1., 0., 2.],
       [1., 5., 0.],
       [0., 2., 1.]])

### A.2.2 COO

However, the most common way to create sparse matrices (and the one we will be using) is the `COO`.

In [18]:
data  = np.array([1, 2, 1, 5, 2, 1])

col = np.array([0, 2, 0, 1, 1, 2])
row = np.array([0, 0, 1, 1, 2, 2])

H = coo_matrix((data, (row, col)), shape=(3, 3))
H.toarray()

array([[1, 0, 2],
       [1, 5, 0],
       [0, 2, 1]])

### A.2.3 Compressed Sparse

Numpy matrices can easily be converted to the `CSR` format, so that we can efficiently operate on them.

In [19]:
# Redefining our first array
data = np.array([1, 0, 2, 1, 5, 0, 0, 2, 1]).reshape(3, 3)

H_ = csr_matrix(data)
H_

<3x3 sparse matrix of type '<class 'numpy.int32'>'
	with 6 stored elements in Compressed Sparse Row format>

In [20]:
H_.data

array([1, 2, 1, 5, 2, 1], dtype=int32)

In [21]:
H_.indptr

array([0, 2, 4, 6], dtype=int32)

In [22]:
H_.indices

array([0, 2, 0, 1, 1, 2], dtype=int32)

The process is exactly the same to convert to `CSC`.

In [23]:
H_ = csc_matrix(data)
H_

<3x3 sparse matrix of type '<class 'numpy.int32'>'
	with 6 stored elements in Compressed Sparse Column format>

Note that, if you already have a sparse matrix in COO format, it can easily be converted to CSR and CSC.

In [24]:
H.tocsr()

<3x3 sparse matrix of type '<class 'numpy.int32'>'
	with 6 stored elements in Compressed Sparse Row format>

In [25]:
H.tocsc()

<3x3 sparse matrix of type '<class 'numpy.int32'>'
	with 6 stored elements in Compressed Sparse Column format>

# B. Vectorization

Python is a high-level language, valuing convenience more than more control over program execution and, ultimately, performance.

Now, as we've seen, recommender systems deal with large amounts of data, thus emphasizing execution and performance.

NumPy (and Pandas, done right) allows us to have both convenience and performance, using *vectorization*. Curious?

Remember the rating matrix $R$, from the previous notebook:

$$\begin{bmatrix}1 &  & 2\\ 1 & 5 & \\  & 2 & 1\end{bmatrix}$$

Let's start by representing it as a list of lists.

In [26]:
R = [[1, nan, 2], [1, 5, nan], [nan, 2, 1]]
R

[[1, nan, 2], [1, 5, nan], [nan, 2, 1]]

Now, we want to compute the mean rating per user.

In [27]:
def mean_user_ratings(R):
    return [mean([r for r in u if not isnan(r)]) for u in R]


mean_user_ratings(R)

[1.5, 3, 1.5]

In this implementation, Python iterates over the 3 rows, plus 3 elements per row, for a total of 9 cycles.

Vectorization, on the other hand, uses Single Instruction Multiple Data ([SIMD](https://en.wikipedia.org/wiki/SIMD), in short) available in most modern CPUs, to:
* Perform an operation
* On multiple data points
* Simultaneously (i.e., in single a cycle).

Vectorization implements what is known as data parallelism, by applying the same transformation to multiple data in parallel.

Again, Python gives us no control over how a program gets executed, to exploit SIMD directly.

The good news is that NumPy implements vectorization for us.

Features such as array methods and universal functions are used to vectorize operations and remove `for` loops.

## B.1 Loading the matrix

Since most times we deal with plain text data, we will use `genfromtxt` to load data from `ratings_matrix.csv`.

In [28]:
R_ = np.genfromtxt('data/interim/ratings_matrix.csv',  delimiter=',')
R_

array([[ 1., nan,  2.],
       [ 1.,  5., nan],
       [nan,  2.,  1.]])

Just to make sure, let's print of the attributes of the array.

In [29]:
ndims = R_.ndim
nrows = R_.shape[0]
ncols = R_.shape[0] 
dtype = R_.dtype

print("R_ is a {}-dimensional, {} by {} matrix, of {} elements.".format(ndims, nrows, ncols, dtype))

R_ is a 2-dimensional, 3 by 3 matrix, of float64 elements.


## B.2 Indexing

### B.2.1 User vectors

Sometimes, we want to select vectors of user ratings, such as $R_u = \begin{bmatrix}r_{u, i_1} & ... & r_{u, i_n}\end{bmatrix}$.

Using the square brackets notation and the row index, we do:

```
    ┌───┬───┬───┐   
    │ 1 │   │ 2 │ 
    ┏━━━┳━━━┳━━━┓          ┏━━━┳━━━┳━━━┓
R = ┃ 1 ┃ 5 ┃   ┃ → R[1] = ┃ 1 ┃ 5 ┃   ┃
    ┗━━━┻━━━┻━━━┛          ┗━━━┻━━━┻━━━┛
    │   │ 2 │ 1 │          (view, shape=(3,))
    └───┴───┴───┘
```

In [30]:
R_[1]

array([ 1.,  5., nan])

This returns a *view* of the original array, meaning that modifying it modifies the base array.

We can check whether or not the resulting array is a view using the attribute `ndarray.base`.

In [31]:
R_[1].base is R_

True

However, regular indexing will return a rank-1 array, i.e. 1-dimensional, which in many cases is not desirable.

In [32]:
R_[1].shape

(3,)

If we want to return a *copy* instead of a view of original array, we need to use advanced indexing.

```
    ┌───┬───┬───┐   
    │ 1 │   │ 2 │ 
    ┏━━━┳━━━┳━━━┓            ┏━━━┳━━━┳━━━┓
R = ┃ 1 ┃ 5 ┃   ┃ → R[[1]] = ┃ 1 ┃ 5 ┃   ┃
    ┗━━━┻━━━┻━━━┛            ┗━━━┻━━━┻━━━┛
    │   │ 2 │ 1 │            (copy, shape=(1, 3))
    └───┴───┴───┘
```

In [33]:
R_[[1]]

array([[ 1.,  5., nan]])

In [34]:
R_[[1]].base is R_

False

Do you see any other difference?

In [35]:
R_[[1]].shape

(1, 3)

Unless you explicitly want to change the array, advanced indexing is recommended.

### B.2.2 Item vectors

We use square brackets with comma separated wildcard and column index.

```
    ┌───┬───┏━━━┓             ┏━━━┓
    │ 1 │   ┃ 2 ┃             ┃ 2 ┃
    ├───┼───┣━━━┫             ┣━━━┫
R = │ 1 │ 5 ┃   ┃ → R[:, 2] = ┃   ┃ (view, shape=(3,))
    ├───┼───┣━━━┫             ┣━━━┫
    │   │ 2 ┃ 1 ┃             ┃ 1 ┃
    └───┴───┗━━━┛             ┗━━━┛
```

In [36]:
R_[:, 2]

array([ 2., nan,  1.])

In [37]:
R_[:, 0].base is R_

True

In [38]:
R_[:, 0].shape

(3,)

Again, we should use fancy indexing:

```
    ┌───┬───┏━━━┓               ┏━━━┓
    │ 1 │   ┃ 2 ┃               ┃ 2 ┃
    ├───┼───┣━━━┫               ┣━━━┫
R = │ 1 │ 5 ┃   ┃ → R[:, [2]] = ┃   ┃ (copy, shape=(3, 1))
    ├───┼───┣━━━┫               ┣━━━┫
    │   │ 2 ┃ 1 ┃               ┃ 1 ┃
    └───┴───┗━━━┛               ┗━━━┛
```

In [39]:
R_[:, [2]]

array([[ 2.],
       [nan],
       [ 1.]])

In [40]:
R_[:, [2]].base is R_

False

In [41]:
R_[:, [2]].shape

(3, 1)

### B.2.3 Ratings

To select individual elements of our ratings matrix, we combine the notations above, so that:

```
    ┏━━━┓───┬───┐             ┏━━━┓
    ┃ 1 ┃   │ 2 │ → R[0, 0] = ┃ 1 ┃ (scalar)
    ┗━━━┛───┼───┤             ┗━━━┛
R = │ 1 │ 5 │   │  
    ├───┼───┼───┤
    │   │ 2 │ 1 │
    └───┴───┴───┘
```

In [42]:
R_[0, 0]

1.0

### B.2.4 Boolean masks

We use boolean arrays to select specific locations according to a condition.

A boolean mask is an array of boolean values.

```
                  ┌───┬───┬───┐
                  │ 0 │ 1 │ 0 │
                  ├───┼───┼───┤                    
M = np.isnan(R) = │ 0 │ 0 │ 1 │ 
                  ├───┼───┼───┤                             
                  │ 1 │ 0 │ 0 │                   
                  └───┴───┴───┘  
```
That can be used

```
    ┌───┬───┬───┐
    │ 1 │   │ 2 │
    ├───┼───┼───┤          ┌───┬───┬───┐
R = │ 1 │ 5 │   │ → R[M] = │   │   │   │
    ├───┼───┼───┤          └───┴───┴───┘       
    │   │ 2 │ 1 │                   
    └───┴───┴───┘                   
```

In [43]:
M = np.isnan(R_)
M

array([[False,  True, False],
       [False, False,  True],
       [ True, False, False]])

In [44]:
R_[M]

array([nan, nan, nan])

### B.2.5 Reshufling vectors

We use these indexing techniques to change the order of the elements of an array.

```
    ┏━━━┓                  ┏━━━┓
    ┃ 0 ┃                  ┃ 1 ┃
    ┣━━━┫                  ┣━━━┫
v = ┃ 1 ┃ → v[[1, 2, 0]] = ┃ 2 ┃
    ┣━━━┫                  ┣━━━┫
    ┃ 2 ┃                  ┃ 0 ┃
    ┗━━━┛                  ┗━━━┛
```

Consider the array `v`. representing a column vector.

In [45]:
v = np.array([[0], [1], [2]])
v

array([[0],
       [1],
       [2]])

We pass an array of indexes to change positions, in a vectorized way.

In [46]:
v[[1, 2, 0]]

array([[1],
       [2],
       [0]])

## B.3 Array methods

If you have done the Prep Course, do you remember numpy array methods? They are pretty handy when dealing with Recommender Systems and Rating Matrixes in general so the following session will mostly be a refresher on these concepts if they are not on your top of mind.

NumPy arrays have many methods ([docs](https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.ndarray.html)), which operate on or with the array and typically return a new array.

Such methods can be grouped according to their purpose:
* Array conversion
* Shape manipulation
* Item selection and manipulation
* Calculation.

Some common operations we can perform along either axis of a `ndarray` are:
* [argmin](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.argmin.html)
* [min](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.min.html)
* [max](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.max.html)
* [round](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.round.html)
* [sum](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.sum.html)
* [cumsum](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.cumsum.html)
* [mean](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.mean.html)
* [var](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.var.html)
* [std](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.std.html)
* [prod](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.prod.html)
* [cumprod](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.cumprod.html)
* [all](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.all.html)
* [any](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.any.html).

We will illustrate a few methods, particularly relevant to our course.

### B.3.1 Reshaping

We can change the shape of the array using `ndarray.reshape`, given the new shape is compatible with the original one.

```
    ┌───┬───┬───┐
    │ 1 │   │ 2 │
    ├───┼───┼───┤                     ┌───┬───┬───┬───┬───┬───┬───┬───┬───┐
R = │ 1 │ 5 │   │ → R.reshape(1, 9) = │ 1 │   │ 2 │ 1 │ 5 │   │   │ 2 │ 1 │
    ├───┼───┼───┤                     └───┴───┴───┴───┴───┴───┴───┴───┴───┘            
    │   │ 2 │ 1 │                   
    └───┴───┴───┘                   
```

We use `reshape` throughout the code to enforce a given shape.

In [47]:
R_.reshape(1, 9)

array([[ 1., nan,  2.,  1.,  5., nan, nan,  2.,  1.]])

### B.3.2 Transposing

Other useful operation is transposing a matrix as in linear algebra.

```
    ┌───┬───┬───┐                   ┌───┬───┬───┐
    │ 1 │   │ 2 │                   │ 1 │ 1 │   │
    ├───┼───┼───┤                   ├───┼───┼───┤
R = │ 1 │ 5 │   │ → R.transpose() = │   │ 5 │ 2 │
    ├───┼───┼───┤                   ├───┼───┼───┤                    
    │   │ 2 │ 1 │                   │ 2 │   │ 1 │
    └───┴───┴───┘                   └───┴───┴───┘
```

In [48]:
R_.transpose()

array([[ 1.,  1., nan],
       [nan,  5.,  2.],
       [ 2., nan,  1.]])

### B.3.3 Argmax

Returns the indices of the maximum values along an axis (0 for columns, 1 for rows).


```
    ┌───┬───┬───┐
    │ 1 │   │ 2 │
    ├───┼───┼───┤                             ┌───┬───┬───┐
R = │ 1 │ 5 │   │ → np.nanargmax(R, axis=1) = │ 2 │ 1 │ 1 │
    ├───┼───┼───┤                             └───┴───┴───┘            
    │   │ 2 │ 1 │                   
    └───┴───┴───┘                   
```

There is, however, a problem: `argmax` returns `np.NaN` values as maximum values.

In [49]:
R_.argmax(axis=1)

array([1, 2, 0], dtype=int64)

Fortunately, NumPy provides a `nanargmax` method to deal with such cases.

In [50]:
np.nanargmax(R_, axis=1)

array([2, 1, 1], dtype=int64)

### B.3.4 Argsort

Returns the indices that would sort an array along an axis.

Unfortunately, NumPy doesn't provide (to our knowledge) a method for missing values.

Also, `argsort` doesn't provide descending order. So we have to improvise!

```
    ┌───┬───┬───┐                        ┌───┬───┬───┐
    │ 1 │   │ 2 │                        │ 1 │ 0 │ 2 │
    ├───┼───┼───┤                        ├───┼───┼───┤
R = │ 1 │ 5 │   │ → R[np.isnan(R)] = 0 → │ 1 │ 5 │ 0 │
    ├───┼───┼───┤                        ├───┼───┼───┤            
    │   │ 2 │ 1 │                        │ 0 │ 2 │ 1 │
    └───┴───┴───┘                        └───┴───┴───┘ 
    
    ┌───┬───┬───┐                        ┌───┬───┬───┐
    │ 1 │ 0 │ 2 │                        │-1 │ 0 │-2 │
    ├───┼───┼───┤                        ├───┼───┼───┤
R = │ 1 │ 5 │ 0 │ → np.negative(R)     = │-1 │-5 │ 0 │
    ├───┼───┼───┤                        ├───┼───┼───┤            
    │ 0 │ 2 │ 1 │                        │ 0 │-2 │-1 │
    └───┴───┴───┘                        └───┴───┴───┘

    ┌───┬───┬───┐                        ┌───┬───┬───┐
    │-1 │ 0 │-2 │                        │ 2 │ 0 │ 1 │
    ├───┼───┼───┤                        ├───┼───┼───┤
R = │-1 │-5 │ 0 │ → R.argsort(axis=1)  = │ 1 │ 0 │ 2 │
    ├───┼───┼───┤                        ├───┼───┼───┤            
    │ 0 │-2 │-1 │                        │ 1 │ 2 │ 0 │
    └───┴───┴───┘                        └───┴───┴───┘ 
```

In [51]:
R_zeros = R_.copy()
R_zeros[np.isnan(R_)] = 0
R_zeros

array([[1., 0., 2.],
       [1., 5., 0.],
       [0., 2., 1.]])

In [52]:
R_zeros = np.negative(R_zeros)
R_zeros

array([[-1., -0., -2.],
       [-1., -5., -0.],
       [-0., -2., -1.]])

In [53]:
R_zeros.argsort(axis=1)

array([[2, 0, 1],
       [1, 0, 2],
       [1, 2, 0]], dtype=int64)

### B.3.5 Mean

Finally, we compute the mean rating per user with vectorization.

However, and this is becoming a common theme, 

The `mean` method doesn't deal with missing values, so we use `np.nanmean`.

```
    ┌───┬───┬───┐                                          ┌───┐
    │ 1 │   │ 2 │                                          │1.5│
    ├───┼───┼───┤                                          ├───┤
R = │ 1 │ 5 │   │ → np.nanmean(R, axis=1, keepdims=True) = │ 3 │
    ├───┼───┼───┤                                          ├───┤      
    │   │ 2 │ 1 │                                          │1.5│
    └───┴───┴───┘                                          └───┘
```

In [54]:
R_.mean(axis=1)

array([nan, nan, nan])

In [55]:
np.nanmean(R_, axis=1, keepdims=True)

array([[1.5],
       [3. ],
       [1.5]])

Not only is vectorization performant, but, thanks to NumPy, it's also expressive.

Many other methods behave just like mean, `min`, `max`, `sum`, `var`, `std`, among others.

## B.4 Universal functions

Additionally to the array methods discussed above, NumPY provides many [universal functions](https://docs.scipy.org/doc/numpy/reference/ufuncs.html#available-ufuncs).

We group the methods, among others, as:
* Math functions
* Trigonometric functions
* Comparison functions.

Many of these, are implemented in compiled C code, for performance.

Because of vectorization, these functions are applied, in parallel, to multiple data at once.

Universal funtions operate element-by-element and include (some them we already know):
* [add](https://docs.scipy.org/doc/numpy/reference/generated/numpy.add.html)
* [subtract](https://docs.scipy.org/doc/numpy/reference/generated/numpy.subtract.html)
* [multiply](https://docs.scipy.org/doc/numpy/reference/generated/numpy.multiply.html)
* [divide](https://docs.scipy.org/doc/numpy/reference/generated/numpy.divide.html)
* [negative](https://docs.scipy.org/doc/numpy/reference/generated/numpy.negative.html)
* [positive](https://docs.scipy.org/doc/numpy/reference/generated/numpy.positive.html)
* [power](https://docs.scipy.org/doc/numpy/reference/generated/numpy.power.html)
* [exp](https://docs.scipy.org/doc/numpy/reference/generated/numpy.exp.html)
* [log](https://docs.scipy.org/doc/numpy/reference/generated/numpy.log.html)
* [sin](https://docs.scipy.org/doc/numpy/reference/generated/numpy.sin.html)
* [cos](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cos.html)
* [greater](https://docs.scipy.org/doc/numpy/reference/generated/numpy.greater.html), [greater_equal](https://docs.scipy.org/doc/numpy/reference/generated/numpy.greater_equal.html)
* [less](https://docs.scipy.org/doc/numpy/reference/generated/numpy.less.html), [less_equal](https://docs.scipy.org/doc/numpy/reference/generated/numpy.less_equal.html)
* [equal](https://docs.scipy.org/doc/numpy/reference/generated/numpy.equal.html)
* [not_equal](https://docs.scipy.org/doc/numpy/reference/generated/numpy.not_equal.html)
* [maximum](https://docs.scipy.org/doc/numpy/reference/generated/numpy.maximum.html)
* [minimum](https://docs.scipy.org/doc/numpy/reference/generated/numpy.minimum.html).

We exemplify some of these functions. Let's start by creating two different matrices.

```
    ┌───┬───┬───┐       ┌───┬───┬───┐
    │ 3 │ 1 │ 2 │       │ 5 │ 5 │ 2 │ 
    ├───┼───┼───┤       ├───┼───┼───┤
A = │ 1 │ 5 │ 4 │   B = │ 3 │ 3 │ 3 │
    ├───┼───┼───┤       ├───┼───┼───┤   
    │ 4 │ 4 │ 1 │       │ 2 │ 5 │ 4 │
    └───┴───┴───┘       └───┴───┴───┘
```

In [56]:
A = np.array([[3, 1, 2], [1, 5, 4], [4, 4, 1]])
B = np.array([[5, 5, 2], [3, 3, 3], [2, 5, 4]])

### B.4.1 Add

We use `add` to add arrays element-wise.

```
┌───┬───┬───┐   ┌───┬───┬───┐   ┌───────┬───────┬───────┐   ┌───┬───┬───┐
│ 3 │ 1 │ 2 │   │ 5 │ 5 │ 2 │   │ 3 + 5 │ 1 + 5 │ 2 + 2 │   │ 8 │ 6 │ 4 │ 
├───┼───┼───┤   ├───┼───┼───┤   ├───────┼───────┼───────┤   ├───┼───┼───┤
│ 1 │ 5 │ 4 │ + │ 3 │ 3 │ 3 │ → │ 1 + 3 │ 5 + 3 │ 4 + 3 │ = │ 4 │ 8 │ 7 │  
├───┼───┼───┤   ├───┼───┼───┤   ├───────┼───────┼───────┤   ├───┼───┼───┤   
│ 4 │ 4 │ 1 │   │ 2 │ 5 │ 4 │   │ 4 + 2 │ 4 + 5 │ 1 + 4 │   │ 6 │ 9 │ 5 │
└───┴───┴───┘   └───┴───┴───┘   └───────┴───────┴───────┘   └───┴───┴───┘
```

In [57]:
np.add(A, B)

array([[8, 6, 4],
       [4, 8, 7],
       [6, 9, 5]])

Alternatively, you can use math operators, as you would normally do (`add` is called internally).

In [58]:
A + B

array([[8, 6, 4],
       [4, 8, 7],
       [6, 9, 5]])

The downside of using math operators is that you need to make sure `A` and `B` are `ndarrays`.

If you use `add` it will convert any input array-like object into a `ndarray` prior to performing the operation.

You can apply the same reasoning to any other math function, as subtract, multiply or divide.

### B.4.2 Log

We use `log` to apply the natural algorithm, element-wise.

```
    ┌───┬───┬───┐               ┌────────┬────────┬────────┐
    │ 3 │ 1 │ 2 │               │ log(3) │ log(1) │ log(2) │
    ├───┼───┼───┤               ├────────┼────────┼────────┤
A = │ 1 │ 5 │ 4 │ → np.log(A) = │ log(1) │ log(5) │ log(4) │
    ├───┼───┼───┤               ├────────┼────────┼────────┤            
    │ 4 │ 4 │ 1 │               │ log(4) │ log(4) │ log(1) │
    └───┴───┴───┘               └────────┴────────┴────────┘
```

In [59]:
np.log(A)

array([[1.09861229, 0.        , 0.69314718],
       [0.        , 1.60943791, 1.38629436],
       [1.38629436, 1.38629436, 0.        ]])

Important functions operate like log, like `exp`, `sin` and `cos`, among others.

### B.4.3 Greater

Compares two arrays element-wise.

```
┌───┬───┬───┐   ┌───┬───┬───┐   ┌───────┬───────┬───────┐   ┌───┬───┬───┐
│ 3 │ 1 │ 2 │   │ 5 │ 5 │ 2 │   │ 3 > 5 │ 1 > 5 │ 2 > 2 │   │ 0 │ 0 │ 0 │ 
├───┼───┼───┤   ├───┼───┼───┤   ├───────┼───────┼───────┤   ├───┼───┼───┤
│ 1 │ 5 │ 4 │ > │ 3 │ 3 │ 3 │ → │ 1 > 3 │ 5 > 3 │ 4 > 3 │ = │ 0 │ 1 │ 1 │  
├───┼───┼───┤   ├───┼───┼───┤   ├───────┼───────┼───────┤   ├───┼───┼───┤   
│ 4 │ 4 │ 1 │   │ 2 │ 5 │ 4 │   │ 4 > 2 │ 4 > 5 │ 1 > 4 │   │ 1 │ 0 │ 0 │
└───┴───┴───┘   └───┴───┴───┘   └───────┴───────┴───────┘   └───┴───┴───┘
```

In [60]:
np.greater(A, B)

array([[False, False, False],
       [False,  True,  True],
       [ True, False, False]])

In [61]:
A > B

array([[False, False, False],
       [False,  True,  True],
       [ True, False, False]])

### B.4.4 Maximum

Returns the maximum of two arrays, element-wise.

```
               ┌───────────┬───────────┬───────────┐   ┌───┬───┬───┐
               │ max(3, 5) │ max(1, 5) │ max(2, 2) │   │ 5 │ 5 │ 2 │ 
               ├───────────┼───────────┼───────────┤   ├───┼───┼───┤
np.max(A, B) → │ max(1, 3) │ max(5, 3) │ max(4, 3) │ = │ 3 │ 5 │ 4 │  
               ├───────────┼───────────┼───────────┤   ├───┼───┼───┤   
               │ max(4, 2) │ max(4, 5) │ max(1, 4) │   │ 4 │ 5 │ 4 │
               └───────────┴───────────┴───────────┘   └───┴───┴───┘
```

In [62]:
np.maximum(A, B)

array([[5, 5, 2],
       [3, 5, 4],
       [4, 5, 4]])

## B.5 Broadcasting

All this convient functionality operates on a element-by-element basis. 

What about operations on arrays with different sizes?

### B.5.1 Columns

We think of boardcasting as streching an array, so that it matches the dimensions of another.

In two dimensions, this means duplicating rows or columns, so that the dimensions of both arrays match.

```
┌───┐   ┌───┬───┬───┐   ┌───┬───┬───┐   ┌───┬───┬───┐
│ 1 │   │ 1 │ 2 │ 3 │   │ 1 │ 1 │ 1 │   │ 1 │ 2 │ 3 │ 
├───┤   ├───┼───┼───┤   ├───┼───┼───┤   ├───┼───┼───┤
│ 2 │ + │ 4 │ 5 │ 6 │ → │ 2 │ 2 │ 2 │ + │ 4 │ 5 │ 6 │   
├───┤   ├───┼───┼───┤   ├───┼───┼───┤   ├───┼───┼───┤   
│ 3 │   │ 7 │ 8 │ 9 │   │ 3 │ 3 │ 3 │   │ 7 │ 8 │ 9 │
└───┘   └───┴───┴───┘   └───┴───┴───┘   └───┴───┴───┘
(3, 1)      (3, 3)          (3, 3)          (3, 3)
```

In this case, we duplicate columns, so that both matrices become the same shape.

If the number of columns of both matrices aren't multiples, an error will be thrown.

In [63]:
A = np.arange(1, 4).reshape(3, 1)
B = np.arange(1, 10).reshape(3, 3)

np.add(A, B)

array([[ 2,  3,  4],
       [ 6,  7,  8],
       [10, 11, 12]])

### B.5.2 Rows

We can do the same to the rows, given that the number of rows in each matrix are multiples.

```
                ┌───┬───┬───┐   ┌───┬───┬───┐   ┌───┬───┬───┐
                │ 1 │ 2 │ 3 │   │ 1 │ 2 │ 3 │   │ 1 │ 2 │ 3 │ 
┌───┬───┬───┐   ├───┼───┼───┤   ├───┼───┼───┤   ├───┼───┼───┤
│ 1 │ 2 │ 3 │ + │ 4 │ 5 │ 6 │ → │ 1 │ 2 │ 3 │ + │ 4 │ 5 │ 6 │   
└───┴───┴───┘   ├───┼───┼───┤   ├───┼───┼───┤   ├───┼───┼───┤   
    (1, 3)      │ 7 │ 8 │ 9 │   │ 1 │ 2 │ 3 │   │ 7 │ 8 │ 9 │
                └───┴───┴───┘   └───┴───┴───┘   └───┴───┴───┘
                    (3, 3)          (3, 3)          (3, 3)
```

In [64]:
A = A.reshape(1, 3)

np.add(A, B)

array([[ 2,  4,  6],
       [ 5,  7,  9],
       [ 8, 10, 12]])

# C Matrix Algebra

In the previous learning unit, we introduced vectorization and data-parallelization, i.e., performing a single operation on multiple data at once.

Vectorization decreases time complexity, grounded in $n$-dimensional arrays and the methods and functions that operate on them.

Vectorized instructions are applied to the data as a whole, avoiding cycles and, particularly, `for` loops.

A particularly relevant set of methods deliberately left out of the last unit, relates to linear algebra and matrix multiplication.

## C.1 Notation

We write $A \in \mathbb{R}^{\space m \space \times \space n}$ a matrix with $m$ rows and $n$ columns, where the entries of $A$ are real numbers.

An $n$-dimensional vector is a vector with $n$ entries, as $x \in \mathbb{R}^n$, represented as a matrix with $n$ rows and one column, i.e., a column-vector.

$$x = \begin{bmatrix} x_0 \\ x_1 \\ ... \\ x_n \end{bmatrix}$$ 

If we want to represent $x$ as a matrix with $n$ columns and a single row, i.e., a row-vector, we explicitly write the vector transposed.

$$x^T = \begin{bmatrix}x_0 & x_1 & ... & x_n \end{bmatrix}$$

We use $a_{ij}$ to represent the value of $A$ in the $i$-th row and the $j$-th column, $a_j$ for the $j$-th column and $a_i^T$ for the $i$-th row.

$$A = \begin{bmatrix}a_{00} & a_{01} & ... & a_{0n} \\ a_{10} & a_{11} & ... & a_{1n} \\ ...  & ... & ... & ...\\ a_{m0} & a_{m1} & ... & a_{mn}\end{bmatrix}$$

$$a_j = \begin{bmatrix} a_{0j} \\ a_{1j} \\ ... \\ a_{mj} \end{bmatrix}$$ 

$$a_i= \begin{bmatrix}a_{i0} & a_{i1} & ... & a_{in} \end{bmatrix}$$

We move to matrix multiplication, the critical matrix operation in memory-based recommender systems.

## C.2 Matrix Multiplication

The product of two matrices $A \in \mathbb{R}^{\space m \space \times \space n}$ and $B \in \mathbb{R}^{\space n \space \times \space p}$ is the matrix $C = AB \in \mathbb{R}^{\space m \space \times \space p}$, where:

$$C_{ij} = \sum\limits_{k=1}^n A_{ik}B_{kj}$$

In short, each value in $C_{ik}$ is the sum of the product of the $i$th-row of $A$ by the $j$-th column of $B$.

For the matrix product to exist, the number of columns in $A$ must equal the number of rows in $B$.

## C.3 Vector-vector products

This product is known as the scalar or *dot* product, from the centered dot that is used to designate this operation, as in $x \cdot y$.

Given $x^T, y \in \mathbb{R}^n$, the inner product or dot-product of the vectors is:

$$x^Ty \in R = \begin{bmatrix}x_0 & x_1 & ... & x_n\end{bmatrix} \begin{bmatrix}y_0 \\ y_1 \\ ... \\ y_n\end{bmatrix} = \sum\limits_{i=1}^n x_{i}y_{i}$$

Note that this is a particular case of matrix multiplication, with a single row and a single column.

This operation is sometimes also referred to as the sum-product, popularized by an Excel function with that name.

In [65]:
x = [0, 1, 2, 3, 4]
y = [5, 6, 7, 8, 9]

Let's implement this in vanilla Python, to build up our intuition. 

We will use `zip` to aggregate both lists element-wise.

In [66]:
list(zip(x, y))

[(0, 5), (1, 6), (2, 7), (3, 8), (4, 9)]

In [67]:
def dot_product(x, y):
    result = 0
    for i, j in zip(x, y):
        result += i*j
        
    return result


dot_product(x, y)

80

It is always the case that $x^Ty = y^Tx$.

Without surprises, NumPy provides us with a vectorized function for the dot-product of two arrays.

We transpose a matrix using the array method `transposed`, or simply `T`.

In [68]:
x_ = np.array(x).reshape(-1, 1)
y_ = np.array(y).reshape(-1, 1)

np.dot(x_.T, y_)

array([[80]])

## C.4 Matrix-Vector products

Given a matrix $A \in \mathbb{R}^{\space m \space \times \space n}$ and a vector $x \in \mathbb{R}^{\space n}$, the product is a vector $y = Ax \in \mathbb{R}^{\space m}$. 

(Perhaps it's worth remembering that $x$ is an $n \times 1$ matrix. Hence, the shape of $y$.)

There are four equivalent ways to look at matrix-vector multiplication. Take $A$ by the rows, we write it as:

$$y = Ax = \begin{bmatrix}a_0^Tx \\ a_1^Tx \\ ... \\ a_m^Tx\end{bmatrix}$$

In other words, the $i$-th entry of $y$ is equal to the inner product of the $i$th row of $A$ and $x$, $y_i = a_i^T x$.

However, if we write $A$ in column form:

$$y = Ax = a_0x_0 + a_1x_1 + ... + a_nx_n$$

Thus, $y$ is a linear combination of the columns of $A$, where $x$ gives the coefficients.

It's also possible to multiply on the left by a row vector. Using the columns from $A$:

$$y^T = x^TA = \begin{bmatrix}x^Ta_0 & x^Ta_1 & ... & x^Ta_n \end{bmatrix}$$

Or, alternatively, using the rows:

$$y^T = x_0a_0^T + x_1a_1^T + ... + x_na_n^T$$

We have that $y^T$ is a linear combination of the rows of $A$, where $x$ is a vector of coefficients.

In [69]:
H = np.arange(9).reshape(3, 3)
H

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [70]:
h = np.arange(3).reshape(-1, 1)
h

array([[0],
       [1],
       [2]])

In [71]:
np.dot(H, h)

array([[ 5],
       [14],
       [23]])

## C.5 Matrix-Matrix products

Using the above, we can build our intuition of the matrix-matrix multiplication $C = AB$.

We start by representing matrix multiplication as a set of dot-products.

$$C = AB = \begin{bmatrix}a_0^Tb_0 & a_0^Tb_1 & ... & a_0^Tb_p \\ a_1^Tb_0 & a_1^Tb_1 & ... & a_1^Tb_p \\ ...  & ... & ... & ...\\ a_m^Tb_0 & a_m^Tb_1 & ... & a_m^Tb_p\end{bmatrix}$$

Remember that since $A \in \mathbb{R}^{\space m \space \times \space n}$ and $B \in \mathbb{R}^{\space n \space \times \space p}$, we have $a_i \in \mathbb{R}^n$ and $b_j \in \mathbb{R}^n$.

A second approach is to represent matrix multiplication as a set of matrix-vector products, each returning a column-vector.

$$C = AB = \begin{bmatrix}Ab_0 & Ab_1 & ... & Ab_p\end{bmatrix}$$

A third one is to use matrix-vector products with the vector on the left, each returning a row-vector.

$$C = AB = \begin{bmatrix}a_0^TB \\ a_1^TB \\ ... \\ a_m^TB\end{bmatrix}$$

In [72]:
I = np.arange(12).reshape(3, 4)
I

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [73]:
np.dot(H, I)

array([[ 20,  23,  26,  29],
       [ 56,  68,  80,  92],
       [ 92, 113, 134, 155]])

Combining sparse matrices with linear algebra, we manage space and time complexity when building memory-based RS.